In [ ]:
import tensorflow_datasets as tfds
import numpy as np

ds = tfds.load('so2sat/rgb', split=('train[:90%]','train[90%:]', 'validation'),batch_size = -1)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling ~/tensorflow_datasets/so2sat/rgb/2.1.0.incompleteSXTRF7/so2sat-train.tfrecord*...:   0%|          | …

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling ~/tensorflow_datasets/so2sat/rgb/2.1.0.incompleteSXTRF7/so2sat-validation.tfrecord*...:   0%|       …

Dataset so2sat downloaded and prepared to ~/tensorflow_datasets/so2sat/rgb/2.1.0. Subsequent calls will reuse this data.


In [ ]:
train = ds[0]
val = ds[1]
test = ds[2]
X_train = train['image']
X_val = val['image']
X_test = test['image']
y_train = train['label']
y_val = val['label']
y_test = val['label']

print(len(X_train))
print(len(X_val))


352366
24119


In [ ]:
import numpy as np
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_val = np.asarray(X_val)
y_val = np.asarray(y_val)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(352366, 32, 32, 3)
(352366,)
(24119, 32, 32, 3)
(24119,)


In [ ]:
import tensorflow as tf
def create_model():
    # Define the model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (5, 5), activation='relu', kernel_initializer='he_uniform', padding='same'),
        tf.keras.layers.Conv2D(32, (5, 5), activation='relu', kernel_initializer='he_uniform', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(128, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(17, activation='softmax')
    ])

    model.compile(optimizer='Adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)
y_test = tf.keras.utils.to_categorical(y_test)
data_train = ImageDataGenerator(rescale=1/255.0).flow(X_train,y_train,batch_size=2048)
data_val = ImageDataGenerator(rescale=1/255.0).flow(X_val,y_val,batch_size=256)
data_test = ImageDataGenerator(rescale=1/255.0).flow(X_test,y_test,batch_size=256)


In [ ]:
import matplotlib.pyplot as plt
def summarize_diagnostics(history):
	# plot loss
	plt.subplot(211)
	plt.title('Cross Entropy Loss')
	plt.plot(history.history['loss'], color='blue', label='train')
	plt.plot(history.history['val_loss'], color='orange', label='test')
	plt.subplot(212)
	plt.title('Classification Accuracy')
	plt.plot(history.history['accuracy'], color='blue', label='train')
	plt.plot(history.history['val_accuracy'], color='orange', label='test')



In [ ]:
model = create_model()
history = model.fit(data_train,epochs=30,validation_data= data_val)
summarize_diagnostics(history)

In [ ]:
from sklearn.metrics import confusion_matrix

#Predict
y_prediction = model.predict(X_test)

#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_prediction , normalize='pred')
print(result)

In [ ]:
!mkdir -p saved_model
model.save('saved_model/SAT_DATA')

In [ ]:
!zip -r /content/SUIM_3.zip /content/saved_model/SUIM_3

In [ ]:
from google.colab import files
files.download("/content/SAT_DATA.zip")